# Patch-based segmentation

Because Monte Carlo validation method for 3D hyperspectral samples causes data leakage between train and test sets, we introduce a new method of validating 3D segmentation.
Instead of randomly picking pixels into sets, we extract _patches_ which are treated as separate images, forming a training set. Patches have fixed size (about 5% of the whole image size) and are drawn randomly until a desired number of pixels is reached (**`TOTAL_NUMBER_OF_TRAIN_SAMPLES`** variable). We have explained this approach in more details in our paper, which can be found [here](https://arxiv.org/abs/1811.03707). 

In [ ]:
import os
from random import randint
from matplotlib import pyplot as plt
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping

from utils import load_patches, combine_patches
from python_research.dataset_structures import HyperspectralDataset, BalancedSubset
from python_research.keras_models import build_3d_model, build_settings_for_dataset, build_1d_model
from python_research.grid_extraction import extract_grids
%matplotlib inline

PATCHES_DIRECTORY = ""
DATA_DIR = os.path.join('..', '..', 'hypernet-data')
RESULTS_DIR = os.path.join('..', '..', 'hypernet-data', 'results', 'grids_validation')
DATASET_PATH = os.path.join(DATA_DIR, 'PaviaU_corrected.npy')
DATASET_GT_PATH = os.path.join(DATA_DIR, 'PaviaU_gt.npy')
os.makedirs(RESULTS_DIR, exist_ok=True)

# Prepare the data

This example allows you to either draw new set of patches to train your model, or use already extracted ones.

The **`PATCH_SIZE`** (WIDTH, HEIGHT) variable lets you define the size of extracted patches.<br/>The **`PIXEL_NEIGHBOURHOOD`** defines the spatial size of each sample. If it is equal to 1, 1D samples will be used, using only the spectral
information of a pixel. If **`PIXEL_NEIGHBOURHOOD`** variable is different than 1, pixel's neighbourhood of size equal to its value will be extracted for each sample. Neighbourhood of the pixel should be provided as an odd number.

In [ ]:
PATCH_SIZE = (17, 30)
PIXEL_NEIGHBOURHOOD = 1
TOTAL_NUMBER_OF_TRAIN_SAMPLES = 2700

# Load data if path to the folder with patches is specified
if PATCHES_DIRECTORY != "":
    train_data, test_data = load_patches(PATCHES_DIRECTORY, PIXEL_NEIGHBOURHOOD)
    dataset_image = test_data.x[:, :, randint(0, test_data.x.shape[-1])]
    train_data.normalize_labels()
    test_data.normalize_labels()
    bands_count = test_data.shape[-1]
    if PIXEL_NEIGHBOURHOOD == 1:
        train_data.expand_dims(axis=-1)
        test_data.expand_dims(axis=-1)
    val_data = BalancedSubset(train_data, 0.1)
# Extract grids from provided dataset if path was not specified
else:
    patches, test_set, dataset_image = extract_grids(DATASET_PATH, DATASET_GT_PATH, PATCH_SIZE, 
                                                     TOTAL_NUMBER_OF_TRAIN_SAMPLES)
    train_data, test_data = combine_patches(patches[0], patches[1], test_set[0], test_set[1], 
                                            PIXEL_NEIGHBOURHOOD)
    train_data.normalize_labels()
    test_data.normalize_labels()
    if PIXEL_NEIGHBOURHOOD == 1:
        train_data.expand_dims(axis=-1)
        test_data.expand_dims(axis=-1)
    val_data = BalancedSubset(train_data, 0.1)

# Patches visualization

Black boxes present pixels (patches) that were extracted into the training and validation sets. In the case of 3D samples, zero-padding was added on the edges of each patch.

In [ ]:
plt.imshow(dataset_image)
plt.show()

# Normalize the data

Data is normalized using Min-Max feature scaling. Min and max values are extracted from train and test sets.

In [ ]:

# Normalize data
max_ = train_data.max if train_data.max > val_data.max else val_data.max
min_ = train_data.min if train_data.min < val_data.min else val_data.min
train_data.normalize_min_max(min_=min_, max_=max_)
val_data.normalize_min_max(min_=min_, max_=max_)
test_data.normalize_min_max(min_=min_, max_=max_)


# Build the model

Build the keras model, depending on the dimensionality of samples prepared earlier.

In [ ]:
CLASSES_COUNT = 9
NUMBER_OF_FILTERS = 200
KERNEL_SIZE = 5
PATIENCE = 15

if PIXEL_NEIGHBOURHOOD == 1:
    model = build_1d_model((test_data.shape[1:]), NUMBER_OF_FILTERS, KERNEL_SIZE, CLASSES_COUNT)
else:
    settings = build_settings_for_dataset((PIXEL_NEIGHBOURHOOD,
                                           PIXEL_NEIGHBOURHOOD))
    model = build_3d_model(settings, CLASSES_COUNT, test_data.shape[-1])

# Callbacks
early = EarlyStopping(patience=PATIENCE)
checkpoint = ModelCheckpoint(os.path.join(RESULTS_DIR, "grids_model"), save_best_only=True)

# Model and data summary

In [ ]:

print(model.summary())
print("Training samples: {}".format(train_data.shape))
print("Validation samples: {}".format(val_data.shape))
print("Test samples: {}".format(test_data.shape))

# Training and evaluation

In [ ]:
EPOCHS = 200
BATCH_SIZE = 64

model.fit(x=train_data.get_data(), 
          y=train_data.get_one_hot_labels(CLASSES_COUNT), 
          batch_size=BATCH_SIZE, 
          epochs=EPOCHS, verbose=False,
          callbacks=[early, checkpoint], 
          validation_data=[val_data.get_data(), val_data.get_one_hot_labels(CLASSES_COUNT)])

best_model = load_model(os.path.join(RESULTS_DIR, "grids_model"))

# Evaluate test set score
accuracy = best_model.evaluate(x=test_data.get_data(), y=test_data.get_one_hot_labels(CLASSES_COUNT))[1]
print("Test set accuracy: {}".format(accuracy))
